# Sentiment-Based Book Recommendation Logic (Non-Interactive)

This notebook demonstrates the core logic of a sentiment aware book recommender system.
It uses TextBlob to analyze review sentiment and suggests alternative books that the user may enjoy.

### Importing Needed Libraries

In [ ]:
import pandas as pd

### Step 1: Load the Data

In [ ]:
df = pd.read_csv(r"C:\book-sentiment-project\data\Book Reviews.csv")
print("Data Loaded")